## AGENDA FOR TODAY

1. Based on current date, figure out what quarters and yearly data are you getting (do this in the intro page)
2. If you don't get the data for a particular stuff, how to handle the error
3. Colbert integration based on it

In [1]:
from src.vectorDatabase import get_earnings_all_quarters_data
from langchain.schema import Document
from src.secData import sec_main

In [2]:
import datetime as dt
import pandas as pd
from datetime import date

quarter = pd.Timestamp(date.today()).quarter
quarter

1

In [3]:
type(pd.Timestamp(date.today()).year)

int

In [4]:
from tenacity import RetryError
def get_data(ticker:str,year:int):
    curr_timestamp = pd.Timestamp(date.today())
    curr_year = curr_timestamp.year
    if year<curr_year:
        docs = []
        # print("Earnings Call Q1")
        # try:
        #     docs,speakers_list_1 = get_earnings_all_quarters_data(docs, "Q1", ticker, year)
        # except RetryError:
        #     print(f"Don't have the data for Q1")
        #     speakers_list_1 = []

        # print("Earnings Call Q2")
        # try:
        #     docs,speakers_list_2 = get_earnings_all_quarters_data(docs, "Q2", ticker, year)
        # except RetryError:
        #     print(f"Don't have the data for Q2")
        #     speakers_list_2 = []
        # print("Earnings Call Q3")
        # try:
        #     docs,speakers_list_3 = get_earnings_all_quarters_data(docs, "Q3", ticker, year)
        # except RetryError:
        #     print(f"Don't have the data for Q3")
        #     speakers_list_3 = []

        print("SEC")
        section_texts,_ = sec_main(ticker, year)
        for filings in section_texts:
            texts_dict = filings[-1]

            for section_name, text in texts_dict.items():
                docs.append(
                    Document(
                        page_content=text,
                        metadata={
                            "accessionNumber": filings[0],
                            "filing_type": filings[1],
                            "filingDate": filings[2],
                            "reportDate": filings[3],
                            "sectionName": section_name,
                        },
                    )
                )
        return docs
    else:
        recent_quarter = curr_timestamp.quarter
        

In [5]:
ticker = 'GOOGL'

In [6]:
docs = get_data(ticker,2023)

SEC
Started Scraping
Scraped
Started Extracting
Extracted


In [7]:
collection = [d.page_content for d in docs]

In [5]:
# import pickle
# with open("test.pkl", "wb") as fp:   #Pickling
#     pickle.dump(collection, fp)
# with open("test.pkl", "rb") as fp:   # Unpickling
#     collection = pickle.load(fp)

FileNotFoundError: [Errno 2] No such file or directory: 'test.pkl'

In [13]:
len(collection)

50

In [28]:
from src.prepline_sec_filings.fetch import get_cik_by_ticker
import requests
import re
from datetime import datetime
import pandas as pd

year = 2021
ticker = 'TSLA'
include_amends = True
filing_types = ['10-K','10-Q']
cik = get_cik_by_ticker(ticker)
rgld_cik = int(cik.strip("0"))

forms = []
if include_amends:
    for ft in filing_types:
        forms.append(ft)
        forms.append(ft+"/A")

url = f"https://data.sec.gov/submissions/CIK{cik}.json"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
# Send a GET request to the URL with headers
response = requests.get(url, headers=headers)

if response.status_code == 200:
    json_data = response.json()
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")

form_lists = []
filings = json_data["filings"]
recent_filings = filings["recent"]
quarter_vals = []
for acc_num, form_name, filing_date, report_date in zip(
    recent_filings["accessionNumber"],
    recent_filings["form"],
    recent_filings["filingDate"],
    recent_filings["reportDate"],
):
    if form_name in forms and report_date.startswith(str(year)):
        if form_name == "10-Q":
            datetime_obj = datetime.strptime(report_date,'%Y-%m-%d')
            quarter = pd.Timestamp(datetime_obj).quarter
            quarter_vals.append(str(quarter))
            form_name += str(quarter)
        no_dashes_acc_num = re.sub("-", "", acc_num)
        form_lists.append([no_dashes_acc_num, form_name, filing_date, report_date])

10-K/A
10-K
10-Q
10-Q
10-Q


In [29]:
tuple(forms)

('10-K', '10-K/A', '10-Q', '10-Q/A')

In [26]:
s = '10-K/A'

s == forms[1]

True

In [20]:
url

'https://data.sec.gov/submissions/CIK0001318605.json'

In [21]:
cik

'0001318605'

In [18]:
from datetime import datetime

d = '2023-10-23'
datetime_obj = datetime.strptime(d,'%Y-%m-%d')
quarter = pd.Timestamp(datetime_obj).quarter
quarter

4

In [1]:
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

No CUDA runtime is found, using CUDA_HOME='/usr'


In [9]:
collection = [doc.page_content for doc in docs][:10]

In [10]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens
max_id = 10000

index_name = f'SEC.{ticker}.{nbits}bits'

In [11]:
import os
os.environ['COLBERT_LOAD_TORCH_EXTENSION_VERBOSE'] = "True"

In [2]:
import torch
torch.cuda.is_available()

False

In [13]:
checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection, overwrite=True)



[Feb 06, 12:29:53] #> Note: Output directory /home/athekunal/Finance LLM/Finance-Data-LLM-Project/experiments/notebook/indexes/SEC.GOOGL.2bits already exists


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen"

WARNING clustering 2018 points to 512 centroids: please provide at least 19968 training points
Using /home/athekunal/.cache/torch_extensions/py38_cu116 as PyTorch extensions root...
Creating extension directory /home/athekunal/.cache/torch_extensions/py38_cu116/decompress_residuals_cpp...


  Iteration 3 (0.09 s, search 0.09 s): objective=231.43 imbalance=1.563 nsplit=0        
[Feb 06, 12:30:00] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


Detected CUDA files, patching ldflags
Emitting ninja build file /home/athekunal/.cache/torch_extensions/py38_cu116/decompress_residuals_cpp/build.ninja...
Building extension module decompress_residuals_cpp...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /usr/bin/nvcc  -DTORCH_EXTENSION_NAME=decompress_residuals_cpp -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/athekunal/anaconda3/envs/colbert/lib/python3.8/site-packages/torch/include -isystem /home/athekunal/anaconda3/envs/colbert/lib/python3.8/site-packages/torch/include/torch/csrc/api/include -isystem /home/athekunal/anaconda3/envs/colbert/lib/python3.8/site-packages/torch/include/TH -isystem /home/athekunal/anaconda3/envs/colbert/lib/python3.8/site-packages/torch/include/THC -isystem /home/athekunal/anaconda3/envs/colbert/include/python3.8 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_86,code=compute_86 -gencode=arch=compute_86,code=sm_86 --compiler-options '-fPIC' -std=c++14 -c /home/athekunal/anaconda3/envs/colbert/lib/p

Process Process-6:
Traceback (most recent call last):
  File "/home/athekunal/anaconda3/envs/colbert/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1900, in _run_ninja_build
    subprocess.run(
  File "/home/athekunal/anaconda3/envs/colbert/lib/python3.8/subprocess.py", line 516, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['ninja', '-v']' returned non-zero exit status 1.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/athekunal/anaconda3/envs/colbert/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/athekunal/anaconda3/envs/colbert/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/athekunal/anaconda3/envs/colbert/lib/python3.8/site-packages/colbert/infra/launcher.py", line 134, in setup_new_process
    return_val = callee(config,

KeyboardInterrupt: 

In [ ]:
indexer.get_index() # You can get the absolute path of the index, if needed.

In [ ]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection)


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

In [ ]:
query = "What did Apple talk about Generative AI?" # try with an in-range query or supply your own
print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

In [3]:
from src.vectorDatabase import get_all_docs

d = get_all_docs("AAPL",2023)

Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
SEC
Started Scraping
Scraped
Started Extracting
Extracted


In [1]:
from prepline_sec_filings.title import is_possible_title

is_possible_title("Hello")

True